Imports

In [1]:
#Base
import pandas as pd
import numpy as np
import os
import datetime

# Data preparation
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.decomposition import TruncatedSVD

Models

In [52]:
#Models
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
# from tensorflow import keras
# from tensorflow.keras import layers

# from keras.models import Sequential
# from keras.layers import Dense
# from sklearn.pipeline import Pipeline
# from scikeras.wrappers import KerasClassifier

Read Data

In [64]:
target_col = ["loan_default"]

In [104]:
data = pd.read_csv("train.csv")
print(f"The shape of Training data is {data.shape}")
# target = data[target_col]
# data = data.drop(target_col, axis=1)
data.head()

The shape of Training data is (233154, 41)


,UniqueID,disbursed_amount,asset_cost,ltv,branch_id,supplier_id,manufacturer_id,Current_pincode_ID,Date.of.Birth,Employment.Type,DisbursalDate,State_ID,Employee_code_ID,MobileNo_Avl_Flag,Aadhar_flag,PAN_flag,VoterID_flag,Driving_flag,Passport_flag,PERFORM_CNS.SCORE,PERFORM_CNS.SCORE.DESCRIPTION,PRI.NO.OF.ACCTS,PRI.ACTIVE.ACCTS,PRI.OVERDUE.ACCTS,PRI.CURRENT.BALANCE,PRI.SANCTIONED.AMOUNT,PRI.DISBURSED.AMOUNT,SEC.NO.OF.ACCTS,SEC.ACTIVE.ACCTS,SEC.OVERDUE.ACCTS,SEC.CURRENT.BALANCE,SEC.SANCTIONED.AMOUNT,SEC.DISBURSED.AMOUNT,PRIMARY.INSTAL.AMT,SEC.INSTAL.AMT,NEW.ACCTS.IN.LAST.SIX.MONTHS,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,AVERAGE.ACCT.AGE,CREDIT.HISTORY.LENGTH,NO.OF_INQUIRIES,loan_default
0,420825,50578,58400,89.55,67,22807,45,1441,01-01-84,Salaried,03-08-18,6,1998,1,1,0,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0,0
1,537409,47145,65550,73.23,67,22807,45,1502,31-07-85,Self employed,26-09-18,6,1998,1,1,0,0,0,0,598,I-Medium Risk,1,1,1,27600,50200,50200,0,0,0,0,0,0,1991,0,0,1,1yrs 11mon,1yrs 11mon,0,1
2,417566,53278,61360,89.63,67,22807,45,1497,24-08-85,Self employed,01-08-18,6,1998,1,1,0,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0,0
3,624493,57513,66113,88.48,67,22807,45,1501,30-12-93,Self employed,26-10-18,6,1998,1,1,0,0,0,0,305,L-Very High Risk,3,0,0,0,0,0,0,0,0,0,0,0,31,0,0,0,0yrs 8mon,1yrs 3mon,1,1
4,539055,52378,60300,88.39,67,22807,45,1495,09-12-77,Self employed,26-09-18,6,1998,1,1,0,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,1,1


EDA

In [73]:
data.columns[data.isna().any()].tolist()

['Employment.Type']

In [67]:
cat = """
asset_cost
Current_pincode_ID
DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS
disbursed_amount
ltv
NEW.ACCTS.IN.LAST.SIX.MONTHS
NO.OF_INQUIRIES
PERFORM_CNS.SCORE
PRI.ACTIVE.ACCTS
PRI.CURRENT.BALANCE
PRI.DISBURSED.AMOUNT
PRI.NO.OF.ACCTS
PRI.OVERDUE.ACCTS
PRI.SANCTIONED.AMOUNT
PRIMARY.INSTAL.AMT
SEC.ACTIVE.ACCTS
SEC.CURRENT.BALANCE
SEC.DISBURSED.AMOUNT
SEC.INSTAL.AMT
SEC.NO.OF.ACCTS
SEC.OVERDUE.ACCTS
SEC.SANCTIONED.AMOUNT
"""

print("','".join(cat.split("\n")))

','asset_cost','Current_pincode','DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS','disbursed_amount','ltv','NEW.ACCTS.IN.LAST.SIX.MONTHS','NO.OF_INQUIRIES','PERFORM_CNS.SCORE','PRI.ACTIVE.ACCTS','PRI.CURRENT.BALANCE','PRI.DISBURSED.AMOUNT','PRI.NO.OF.ACCTS','PRI.OVERDUE.ACCTS','PRI.SANCTIONED.AMOUNT','PRIMARY.INSTAL.AMT','SEC.ACTIVE.ACCTS','SEC.CURRENT.BALANCE','SEC.DISBURSED.AMOUNT','SEC.INSTAL.AMT','SEC.NO.OF.ACCTS','SEC.OVERDUE.ACCTS','SEC.SANCTIONED.AMOUNT','


Removing columns

In [105]:
remove_cols = ['branch_id','Current_pincode_ID','Employee_code_ID','UniqueID']

data = data.drop(remove_cols, axis=1)

print("Shape of data after removing non-necessary columns: ", data.shape)


Shape of data after removing non-necessary columns:  (233154, 37)


Feature Extraction & Cleaning

In [106]:
def get_age_from_dob(row):
  dob = row['Date.of.Birth']
  disb = row['DisbursalDate']

  dob_year = int(dob.split("-")[2])
  if dob_year < 49:
    dob_year = 2000 + dob_year
  else:
    dob_year = 1900 + dob_year
  
  disb_year = 2000 + int(disb.split("-")[2])

  age_at_disbursement = disb_year - dob_year
  return age_at_disbursement


def get_avg_acc_age_in_months(row):
  avg_age = row['AVERAGE.ACCT.AGE']
  yrs, mth = avg_age.split(" ")[0], avg_age.split(" ")[1]
  yrs = int(yrs.replace("yrs",""))
  mth = int(mth.replace("mon",""))

  total_months = yrs*12 + mth
  return total_months

def get_credit_age_in_months(row):
  avg_age = row['CREDIT.HISTORY.LENGTH']
  yrs, mth = avg_age.split(" ")[0], avg_age.split(" ")[1]
  yrs = int(yrs.replace("yrs",""))
  mth = int(mth.replace("mon",""))

  total_months = yrs*12 + mth
  return total_months

def clean_employment_type(row):
  if row['Employment.Type'] not in ['Self employed', 'Salaried']:
    return 'NOT Defined'
  else:
    return row['Employment.Type']

  yrs, mth = avg_age.split(" ")[0], avg_age.split(" ")[1]
  yrs = int(yrs.replace("yrs",""))
  mth = int(mth.replace("mon",""))

  total_months = yrs*12 + mth
  return total_months

data["age_at_disbursement"] = data.apply (lambda row: get_age_from_dob(row), axis=1)
data["AVERAGE.ACCT.AGE_Months"] = data.apply(lambda row: get_avg_acc_age_in_months(row), axis=1)
data["credit_history_in_months"] = data.apply (lambda row: get_credit_age_in_months(row), axis=1)
data["Employment.Type"] = data.apply (lambda row: clean_employment_type(row), axis=1)

data = data.drop(['DisbursalDate','Date.of.Birth','AVERAGE.ACCT.AGE','CREDIT.HISTORY.LENGTH'], axis=1)

print("Shape of data after feature extraction: ", data.shape)
data.head()

Shape of data after feature extraction:  (233154, 36)


,disbursed_amount,asset_cost,ltv,supplier_id,manufacturer_id,Employment.Type,State_ID,MobileNo_Avl_Flag,Aadhar_flag,PAN_flag,VoterID_flag,Driving_flag,Passport_flag,PERFORM_CNS.SCORE,PERFORM_CNS.SCORE.DESCRIPTION,PRI.NO.OF.ACCTS,PRI.ACTIVE.ACCTS,PRI.OVERDUE.ACCTS,PRI.CURRENT.BALANCE,PRI.SANCTIONED.AMOUNT,PRI.DISBURSED.AMOUNT,SEC.NO.OF.ACCTS,SEC.ACTIVE.ACCTS,SEC.OVERDUE.ACCTS,SEC.CURRENT.BALANCE,SEC.SANCTIONED.AMOUNT,SEC.DISBURSED.AMOUNT,PRIMARY.INSTAL.AMT,SEC.INSTAL.AMT,NEW.ACCTS.IN.LAST.SIX.MONTHS,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,NO.OF_INQUIRIES,loan_default,age_at_disbursement,AVERAGE.ACCT.AGE_Months,credit_history_in_months
0,50578,58400,89.55,22807,45,Salaried,6,1,1,0,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,34,0,0
1,47145,65550,73.23,22807,45,Self employed,6,1,1,0,0,0,0,598,I-Medium Risk,1,1,1,27600,50200,50200,0,0,0,0,0,0,1991,0,0,1,0,1,33,23,23
2,53278,61360,89.63,22807,45,Self employed,6,1,1,0,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,33,0,0
3,57513,66113,88.48,22807,45,Self employed,6,1,1,0,0,0,0,305,L-Very High Risk,3,0,0,0,0,0,0,0,0,0,0,0,31,0,0,0,1,1,25,8,15
4,52378,60300,88.39,22807,45,Self employed,6,1,1,0,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,41,0,0


One Hot Encoding





In [107]:
categorical_cols = ['Aadhar_flag','Driving_flag','Employment.Type',
                    'manufacturer_id','MobileNo_Avl_Flag','PAN_flag','Passport_flag',
                    'PERFORM_CNS.SCORE.DESCRIPTION','VoterID_flag'
]
for i in categorical_cols:
    cols = pd.get_dummies(data[i], prefix=i, drop_first=False)
    data = pd.concat([data, cols], axis=1)    

data = data.drop(categorical_cols, axis=1)
print("Shape of data after one-hot encoding: ", data.shape)
data.head()

Shape of data after one-hot encoding:  (233154, 72)


,disbursed_amount,asset_cost,ltv,supplier_id,State_ID,PERFORM_CNS.SCORE,PRI.NO.OF.ACCTS,PRI.ACTIVE.ACCTS,PRI.OVERDUE.ACCTS,PRI.CURRENT.BALANCE,PRI.SANCTIONED.AMOUNT,PRI.DISBURSED.AMOUNT,SEC.NO.OF.ACCTS,SEC.ACTIVE.ACCTS,SEC.OVERDUE.ACCTS,SEC.CURRENT.BALANCE,SEC.SANCTIONED.AMOUNT,SEC.DISBURSED.AMOUNT,PRIMARY.INSTAL.AMT,SEC.INSTAL.AMT,NEW.ACCTS.IN.LAST.SIX.MONTHS,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,NO.OF_INQUIRIES,loan_default,age_at_disbursement,AVERAGE.ACCT.AGE_Months,credit_history_in_months,Aadhar_flag_0,Aadhar_flag_1,Driving_flag_0,Driving_flag_1,Employment.Type_NOT Defined,Employment.Type_Salaried,Employment.Type_Self employed,manufacturer_id_45,manufacturer_id_48,manufacturer_id_49,manufacturer_id_51,manufacturer_id_67,manufacturer_id_86,manufacturer_id_120,manufacturer_id_145,manufacturer_id_152,manufacturer_id_153,manufacturer_id_156,MobileNo_Avl_Flag_1,PAN_flag_0,PAN_flag_1,Passport_flag_0,Passport_flag_1,PERFORM_CNS.SCORE.DESCRIPTION_A-Very Low Risk,PERFORM_CNS.SCORE.DESCRIPTION_B-Very Low Risk,PERFORM_CNS.SCORE.DESCRIPTION_C-Very Low Risk,PERFORM_CNS.SCORE.DESCRIPTION_D-Very Low Risk,PERFORM_CNS.SCORE.DESCRIPTION_E-Low Risk,PERFORM_CNS.SCORE.DESCRIPTION_F-Low Risk,PERFORM_CNS.SCORE.DESCRIPTION_G-Low Risk,PERFORM_CNS.SCORE.DESCRIPTION_H-Medium Risk,PERFORM_CNS.SCORE.DESCRIPTION_I-Medium Risk,PERFORM_CNS.SCORE.DESCRIPTION_J-High Risk,PERFORM_CNS.SCORE.DESCRIPTION_K-High Risk,PERFORM_CNS.SCORE.DESCRIPTION_L-Very High Risk,PERFORM_CNS.SCORE.DESCRIPTION_M-Very High Risk,PERFORM_CNS.SCORE.DESCRIPTION_No Bureau History Available,PERFORM_CNS.SCORE.DESCRIPTION_Not Scored: More than 50 active Accounts found,PERFORM_CNS.SCORE.DESCRIPTION_Not Scored: No Activity seen on the customer (Inactive),PERFORM_CNS.SCORE.DESCRIPTION_Not Scored: No Updates available in last 36 months,PERFORM_CNS.SCORE.DESCRIPTION_Not Scored: Not Enough Info available on the customer,PERFORM_CNS.SCORE.DESCRIPTION_Not Scored: Only a Guarantor,PERFORM_CNS.SCORE.DESCRIPTION_Not Scored: Sufficient History Not Available,VoterID_flag_0,VoterID_flag_1
0,50578,58400,89.55,22807,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,34,0,0,0,1,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0
1,47145,65550,73.23,22807,6,598,1,1,1,27600,50200,50200,0,0,0,0,0,0,1991,0,0,1,0,1,33,23,23,0,1,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
2,53278,61360,89.63,22807,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,33,0,0,0,1,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0
3,57513,66113,88.48,22807,6,305,3,0,0,0,0,0,0,0,0,0,0,0,31,0,0,0,1,1,25,8,15,0,1,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
4,52378,60300,88.39,22807,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,41,0,0,0,1,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0


Scaling

In [110]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scale_cols = ['asset_cost','DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS','disbursed_amount','ltv',
              'NEW.ACCTS.IN.LAST.SIX.MONTHS','NO.OF_INQUIRIES','PERFORM_CNS.SCORE','PRI.ACTIVE.ACCTS','PRI.CURRENT.BALANCE',
              'PRI.DISBURSED.AMOUNT','PRI.NO.OF.ACCTS','PRI.OVERDUE.ACCTS','PRI.SANCTIONED.AMOUNT','PRIMARY.INSTAL.AMT',
              'SEC.ACTIVE.ACCTS','SEC.CURRENT.BALANCE',
              'SEC.DISBURSED.AMOUNT','SEC.INSTAL.AMT','SEC.NO.OF.ACCTS','SEC.OVERDUE.ACCTS','SEC.SANCTIONED.AMOUNT']
# scaler = StandardScaler()
col_list = data.columns.values.tolist()
print(col_list)
data[scale_cols] = scaler.fit_transform(data[scale_cols])
data_n = scaler.fit_transform(data)
data_n = pd.DataFrame(data_n, columns = col_list)


['disbursed_amount', 'asset_cost', 'ltv', 'supplier_id', 'State_ID', 'PERFORM_CNS.SCORE', 'PRI.NO.OF.ACCTS', 'PRI.ACTIVE.ACCTS', 'PRI.OVERDUE.ACCTS', 'PRI.CURRENT.BALANCE', 'PRI.SANCTIONED.AMOUNT', 'PRI.DISBURSED.AMOUNT', 'SEC.NO.OF.ACCTS', 'SEC.ACTIVE.ACCTS', 'SEC.OVERDUE.ACCTS', 'SEC.CURRENT.BALANCE', 'SEC.SANCTIONED.AMOUNT', 'SEC.DISBURSED.AMOUNT', 'PRIMARY.INSTAL.AMT', 'SEC.INSTAL.AMT', 'NEW.ACCTS.IN.LAST.SIX.MONTHS', 'DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS', 'NO.OF_INQUIRIES', 'loan_default', 'age_at_disbursement', 'AVERAGE.ACCT.AGE_Months', 'credit_history_in_months', 'Aadhar_flag_0', 'Aadhar_flag_1', 'Driving_flag_0', 'Driving_flag_1', 'Employment.Type_NOT Defined', 'Employment.Type_Salaried', 'Employment.Type_Self employed', 'manufacturer_id_45', 'manufacturer_id_48', 'manufacturer_id_49', 'manufacturer_id_51', 'manufacturer_id_67', 'manufacturer_id_86', 'manufacturer_id_120', 'manufacturer_id_145', 'manufacturer_id_152', 'manufacturer_id_153', 'manufacturer_id_156', 'MobileNo_Avl

In [112]:
# data_n.head()
data = data_n

Splitting Data

In [113]:
target = data[target_col]
data = data.drop(target_col, axis=1)
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.33, random_state=42, stratify=target)

print("Shape of data:", data.shape)
print("Shape of target", target.shape)
print("Shape of Xtrain and ytrain:", X_train.shape, y_train.shape)
print("Shape of Xtest and ytest:", X_test.shape, y_test.shape)

Shape of data: (233154, 71)
Shape of target (233154, 1)
Shape of Xtrain and ytrain: (156213, 71) (156213, 1)
Shape of Xtest and ytest: (76941, 71) (76941, 1)


In [114]:
X_train.head()

,disbursed_amount,asset_cost,ltv,supplier_id,State_ID,PERFORM_CNS.SCORE,PRI.NO.OF.ACCTS,PRI.ACTIVE.ACCTS,PRI.OVERDUE.ACCTS,PRI.CURRENT.BALANCE,PRI.SANCTIONED.AMOUNT,PRI.DISBURSED.AMOUNT,SEC.NO.OF.ACCTS,SEC.ACTIVE.ACCTS,SEC.OVERDUE.ACCTS,SEC.CURRENT.BALANCE,SEC.SANCTIONED.AMOUNT,SEC.DISBURSED.AMOUNT,PRIMARY.INSTAL.AMT,SEC.INSTAL.AMT,NEW.ACCTS.IN.LAST.SIX.MONTHS,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,NO.OF_INQUIRIES,age_at_disbursement,AVERAGE.ACCT.AGE_Months,credit_history_in_months,Aadhar_flag_0,Aadhar_flag_1,Driving_flag_0,Driving_flag_1,Employment.Type_NOT Defined,Employment.Type_Salaried,Employment.Type_Self employed,manufacturer_id_45,manufacturer_id_48,manufacturer_id_49,manufacturer_id_51,manufacturer_id_67,manufacturer_id_86,manufacturer_id_120,manufacturer_id_145,manufacturer_id_152,manufacturer_id_153,manufacturer_id_156,MobileNo_Avl_Flag_1,PAN_flag_0,PAN_flag_1,Passport_flag_0,Passport_flag_1,PERFORM_CNS.SCORE.DESCRIPTION_A-Very Low Risk,PERFORM_CNS.SCORE.DESCRIPTION_B-Very Low Risk,PERFORM_CNS.SCORE.DESCRIPTION_C-Very Low Risk,PERFORM_CNS.SCORE.DESCRIPTION_D-Very Low Risk,PERFORM_CNS.SCORE.DESCRIPTION_E-Low Risk,PERFORM_CNS.SCORE.DESCRIPTION_F-Low Risk,PERFORM_CNS.SCORE.DESCRIPTION_G-Low Risk,PERFORM_CNS.SCORE.DESCRIPTION_H-Medium Risk,PERFORM_CNS.SCORE.DESCRIPTION_I-Medium Risk,PERFORM_CNS.SCORE.DESCRIPTION_J-High Risk,PERFORM_CNS.SCORE.DESCRIPTION_K-High Risk,PERFORM_CNS.SCORE.DESCRIPTION_L-Very High Risk,PERFORM_CNS.SCORE.DESCRIPTION_M-Very High Risk,PERFORM_CNS.SCORE.DESCRIPTION_No Bureau History Available,PERFORM_CNS.SCORE.DESCRIPTION_Not Scored: More than 50 active Accounts found,PERFORM_CNS.SCORE.DESCRIPTION_Not Scored: No Activity seen on the customer (Inactive),PERFORM_CNS.SCORE.DESCRIPTION_Not Scored: No Updates available in last 36 months,PERFORM_CNS.SCORE.DESCRIPTION_Not Scored: Not Enough Info available on the customer,PERFORM_CNS.SCORE.DESCRIPTION_Not Scored: Only a Guarantor,PERFORM_CNS.SCORE.DESCRIPTION_Not Scored: Sufficient History Not Available,VoterID_flag_0,VoterID_flag_1
156376,0.022327,0.019888,0.493821,0.848309,0.238095,0.000000,0.000000,0.000000,0.0,0.064710,0.000000,0.000000,0.0,0.0,0.0,0.015698,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.431373,0.00000,0.000000,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
155046,0.034064,0.017596,0.750853,0.259822,0.142857,0.926966,0.002208,0.000000,0.0,0.064710,0.000000,0.000000,0.0,0.0,0.0,0.015698,0.0,0.0,0.000038,0.0,0.000000,0.0,0.000000,0.352941,0.01626,0.012821,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
162500,0.043939,0.035094,0.616924,0.237342,0.095238,0.000000,0.000000,0.000000,0.0,0.064710,0.000000,0.000000,0.0,0.0,0.0,0.015698,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.235294,0.00000,0.000000,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
127710,0.040172,0.018442,0.848535,0.860915,0.238095,0.000000,0.000000,0.000000,0.0,0.064710,0.000000,0.000000,0.0,0.0,0.0,0.015698,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.745098,0.00000,0.000000,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
209953,0.034540,0.021842,0.718254,0.528959,0.142857,0.926966,0.006623,0.006944,0.0,0.065103,0.000042,0.000042,0.0,0.0,0.0,0.015698,0.0,0.0,0.000206,0.0,0.028571,0.0,0.083333,0.529412,0.03252,0.051282,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


XGBoost

In [115]:
from sklearn import metrics

xgb = XGBClassifier()
xgb.fit(X_train, y_train)

xgb_y_pred = xgb.predict(X_test)

cnf_matrix = metrics.confusion_matrix(y_test, xgb_y_pred)
print(cnf_matrix)


print("Accuracy:",metrics.accuracy_score(y_test, xgb_y_pred))
print("Precision:",metrics.precision_score(y_test, xgb_y_pred))
print("Recall:",metrics.recall_score(y_test, xgb_y_pred))

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[[60185    54]
 [16646    56]]
Accuracy: 0.7829505725166036
Precision: 0.509090909090909
Recall: 0.003352891869237217
